In [1]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import gc
import warnings
from tqdm import tqdm
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, KFold
from itertools import product
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.preprocessing import LabelEncoder
from scipy import stats
from feature_selector import FeatureSelector
from sklearn.decomposition import NMF

warnings.filterwarnings('ignore')

pd.set_option('display.unicode.ambiguous_as_wide', True)
pd.set_option('display.unicode.east_asian_width', True)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option("display.max_colwidth", 100)
pd.set_option('display.width', 1000)

C:\Users\z\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
# 节省内存读文件
def reduce_mem_usage(df):
    """
    iterate through all the columns of a dataframe and modify the data type to reduce memory usage.
    @param df:
    @return:
    """
    start_mem = df.memory_usage().sum()
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('str')

    end_mem = df.memory_usage().sum()
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    return df

In [3]:
class MeanEncoder:
    def __init__(self, categorical_features, n_splits=10, target_type='classification', prior_weight_func=None):
        """
        :param categorical_features: list of str, the name of the categorical columns to encode
 
        :param n_splits: the number of splits used in mean encoding
 
        :param target_type: str, 'regression' or 'classification'
 
        :param prior_weight_func:
        a function that takes in the number of observations, and outputs prior weight
        when a dict is passed, the default exponential decay function will be used:
        k: the number of observations needed for the posterior to be weighted equally as the prior
        f: larger f --> smaller slope
        """
 
        self.categorical_features = categorical_features
        self.n_splits = n_splits
        self.learned_stats = {}
 
        if target_type == 'classification':
            self.target_type = target_type
            self.target_values = []
        else:
            self.target_type = 'regression'
            self.target_values = None
 
        if isinstance(prior_weight_func, dict):
            self.prior_weight_func = eval('lambda x: 1 / (1 + np.exp((x - k) / f))', dict(prior_weight_func, np=np))
        elif callable(prior_weight_func):
            self.prior_weight_func = prior_weight_func
        else:
            self.prior_weight_func = lambda x: 1 / (1 + np.exp((x - 2) / 1))
 
    @staticmethod
    def mean_encode_subroutine(X_train, y_train, X_test, variable, target, prior_weight_func):
        X_train = X_train[[variable]].copy()
        X_test = X_test[[variable]].copy()
 
        if target is not None:
            nf_name = '{}_pred_{}'.format(variable, target)
            X_train['pred_temp'] = (y_train == target).astype(int)  # classification
        else:
            nf_name = '{}_pred'.format(variable)
            X_train['pred_temp'] = y_train  # regression
        prior = X_train['pred_temp'].mean()
 
        col_avg_y = X_train.groupby(by=variable, axis=0)['pred_temp'].agg(['mean', 'size'])
        col_avg_y['size'] = prior_weight_func(col_avg_y['size'])
        col_avg_y[nf_name] = col_avg_y['size'] * prior + (1 - col_avg_y['size']) * col_avg_y['mean']
        col_avg_y.drop(['size', 'mean'], axis=1, inplace=True)
 
        nf_train = X_train.join(col_avg_y, on=variable)[nf_name].values
        nf_test = X_test.join(col_avg_y, on=variable).fillna(prior, inplace=False)[nf_name].values
 
        return nf_train, nf_test, prior, col_avg_y
 
    def fit_transform(self, X, y):
        """
        :param X: pandas DataFrame, n_samples * n_features
        :param y: pandas Series or numpy array, n_samples
        :return X_new: the transformed pandas DataFrame containing mean-encoded categorical features
        """
        X_new = X.copy()
        if self.target_type == 'classification':
            skf = StratifiedKFold(self.n_splits)
        else:
            skf = KFold(self.n_splits)
 
        if self.target_type == 'classification':
            self.target_values = sorted(set(y))
            self.learned_stats = {'{}_pred_{}'.format(variable, target): [] for variable, target in
                                  product(self.categorical_features, self.target_values)}
            for variable, target in product(self.categorical_features, self.target_values):
                nf_name = '{}_pred_{}'.format(variable, target)
                X_new.loc[:, nf_name] = np.nan
                for large_ind, small_ind in skf.split(X, y):
                    nf_large, nf_small, prior, col_avg_y = MeanEncoder.mean_encode_subroutine(
                        X_new.iloc[large_ind], y.iloc[large_ind], X_new.iloc[small_ind], variable, target, self.prior_weight_func)
                    X_new.iloc[small_ind, -1] = nf_small
                    self.learned_stats[nf_name].append((prior, col_avg_y))
        else:
            self.learned_stats = {'{}_pred'.format(variable): [] for variable in self.categorical_features}
            for variable in self.categorical_features:
                nf_name = '{}_pred'.format(variable)
                X_new.loc[:, nf_name] = np.nan
                for large_ind, small_ind in skf.split(X, y):
                    nf_large, nf_small, prior, col_avg_y = MeanEncoder.mean_encode_subroutine(
                        X_new.iloc[large_ind], y.iloc[large_ind], X_new.iloc[small_ind], variable, None, self.prior_weight_func)
                    X_new.iloc[small_ind, -1] = nf_small
                    self.learned_stats[nf_name].append((prior, col_avg_y))
        return X_new
 
    def transform(self, X):
        """
        :param X: pandas DataFrame, n_samples * n_features
        :return X_new: the transformed pandas DataFrame containing mean-encoded categorical features
        """
        X_new = X.copy()
 
        if self.target_type == 'classification':
            for variable, target in product(self.categorical_features, self.target_values):
                nf_name = '{}_pred_{}'.format(variable, target)
                X_new[nf_name] = 0
                for prior, col_avg_y in self.learned_stats[nf_name]:
                    X_new[nf_name] += X_new[[variable]].join(col_avg_y, on=variable).fillna(prior, inplace=False)[
                        nf_name]
                X_new[nf_name] /= self.n_splits
        else:
            for variable in self.categorical_features:
                nf_name = '{}_pred'.format(variable)
                X_new[nf_name] = 0
                for prior, col_avg_y in self.learned_stats[nf_name]:
                    X_new[nf_name] += X_new[[variable]].join(col_avg_y, on=variable).fillna(prior, inplace=False)[
                        nf_name]
                X_new[nf_name] /= self.n_splits
 
        return X_new

In [4]:
def count_encode(df, cols):
    for col in tqdm(cols):
        print(col)
        vc = df[col].value_counts(dropna=True, normalize=True)
        df[col + '_count'] = df[col].map(vc).astype('float32')
    return df
        
        
def label_encode(df, cols):
    le = LabelEncoder()
    for col in tqdm(cols):
        df[col] = df[col].fillna('NA')
        df[col] = le.fit_transform(df[col].astype(str))
    return df


def cross_cat_num(df, cat_col, num_col):
    for f1 in tqdm(cat_col):
        g = df.groupby(f1, as_index=False)
        for f2 in tqdm(num_col):
            df_new = g[f2].agg({
                '{}_{}_max'.format(f1, f2): 'max',
                '{}_{}_min'.format(f1, f2): 'min',
                '{}_{}_median'.format(f1, f2): 'median',
                '{}_{}_mean'.format(f1, f2): 'mean',
                '{}_{}_skew'.format(f1, f2): 'skew',
                '{}_{}_nunique'.format(f1, f2): 'nunique'
            })
            df = df.merge(df_new, on=f1, how='left')
            del df_new
            gc.collect()
    return df

## user表

In [5]:
train_user = pd.read_hdf('../input/train_user.h5')
test_user = pd.read_hdf('../input/test_user.h5')

In [6]:
# 将city_name和county_name拼接起来
train_user['city_name_county_name'] = train_user['city_name'].astype(str) + '_' + train_user['county_name'].astype(str)
test_user['city_name_county_name'] = test_user['city_name'].astype(str) + '_' + test_user['county_name'].astype(str)

In [7]:
cat_feat = ['city_name', 'county_name', 'city_name_county_name']
num_feat = ['idcard_cnt', 'arpu_202004', 'idcard_cnt*arpu_202004', 'arpu_202004/idcard_cnt']

In [8]:
y = train_user['label']

In [9]:
ME = MeanEncoder(categorical_features=cat_feat,
                 n_splits=3,
                 target_type='classification',
                 prior_weight_func=None)
X_data = ME.fit_transform(train_user.drop('label', axis=1), y)
X_test = ME.transform(test_user)

In [10]:
train_user = X_data.copy()
train_user['label'] = y
test_user = X_test.copy()

del X_data, X_test
gc.collect()

train_user.shape, test_user.shape

((6106, 13), (2045, 12))

In [11]:
df_user = pd.concat([train_user, test_user])

del train_user, test_user
gc.collect()

20

In [12]:
# 电话的数量*月消费额
df_user['idcard_cnt*arpu_202004'] = df_user['idcard_cnt'] * df_user['arpu_202004']

# 月消费额/电话的数量
df_user['arpu_202004/idcard_cnt'] = df_user['arpu_202004'] / (df_user['idcard_cnt'] + 0.0001)

In [13]:
df_user = cross_cat_num(df_user, cat_feat, num_feat)

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  2.18it/s]


In [14]:
df_user = count_encode(df_user, cat_feat + ['idcard_cnt'])

  0%|                                                                                            | 0/4 [00:00<?, ?it/s]

city_name
county_name
city_name_county_name
idcard_cnt


100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 117.72it/s]


In [15]:
df_user = label_encode(df_user, cat_feat)

100%|███████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 120.06it/s]


In [16]:
df_user.to_hdf('../input/user_features.h5', 'df', index=False)

del df_user
gc.collect()

77

## voc表

In [17]:
train_voc = pd.read_hdf('../input/train_voc.h5')
test_voc = pd.read_hdf('../input/test_voc.h5')

In [18]:
df_voc = pd.concat([train_voc, test_voc])

del train_voc, test_voc
gc.collect()

128

In [19]:
df_voc['city_name_county_name'] = df_voc['city_name'] + '_' + df_voc['county_name']

In [20]:
df_voc['voc_day'] = df_voc['start_datetime'].astype('datetime64').dt.day
df_voc['voc_hour'] = df_voc['start_datetime'].astype('datetime64').dt.hour
df_voc['voc_dayofweek'] = df_voc['start_datetime'].astype('datetime64').dt.dayofweek

In [21]:
phone_no_m = df_voc[['phone_no_m']].copy()
phone_no_m = phone_no_m.drop_duplicates(subset=['phone_no_m'], keep='last')

In [22]:
# 通话次数，通话人数
tmp = df_voc.groupby('phone_no_m')['opposite_no_m'].agg(opposite_cnt='count', opposite_nunique='nunique')
phone_no_m = phone_no_m.merge(tmp, how='left', on='phone_no_m')

# 人均通话次数
phone_no_m['voc_cnt_per_capita'] = phone_no_m['opposite_cnt'] / (phone_no_m['opposite_nunique'] + 0.0001)

del tmp
gc.collect()

20

In [23]:
"""
主叫通话
"""

df_calltype_id_1 = df_voc.loc[df_voc['calltype_id'] == 1, :].copy()

# 主叫通话次数，主叫通话使用的手机个数
tmp = df_calltype_id_1.groupby('phone_no_m')['imei_m'].agg(voc_calltype_id_1_cnt='count', imeis_1='nunique')
phone_no_m = phone_no_m.merge(tmp, on='phone_no_m', how='left')
del tmp
gc.collect()

# 主叫通话时长
tmp = df_calltype_id_1.groupby('phone_no_m')['call_dur'].agg(voc_calltype_id_1_call_dur_sum='sum')
phone_no_m = phone_no_m.merge(tmp, on='phone_no_m', how='left')
del tmp
gc.collect()

# 主叫通话时长小于10s的次数
# tmp1 = df_calltype_id_1[df_calltype_id_1['call_dur'] < 10]
# tmp2 = tmp1.groupby('phone_no_m')['call_dur'].agg(voc_calltype_id_1_10s_cnt='count')
# phone_no_m = phone_no_m.merge(tmp2, on='phone_no_m', how='left')
# del tmp1, tmp2
# gc.collect()

# 主叫通话时长小于30s的次数
tmp1 = df_calltype_id_1[df_calltype_id_1['call_dur'] < 30]
tmp2 = tmp1.groupby('phone_no_m')['call_dur'].agg(voc_calltype_id_1_30s_cnt='count')
phone_no_m = phone_no_m.merge(tmp2, on='phone_no_m', how='left')
del tmp1, tmp2
gc.collect()

# 主叫通话时长小于60s的次数
tmp1 = df_calltype_id_1[df_calltype_id_1['call_dur'] < 60]
tmp2 = tmp1.groupby('phone_no_m')['call_dur'].agg(voc_calltype_id_1_60s_cnt='count')
phone_no_m = phone_no_m.merge(tmp2, on='phone_no_m', how='left')
del tmp1, tmp2
gc.collect()

# 主叫通话时长大于300s（5分钟）的次数
tmp1 = df_calltype_id_1[df_calltype_id_1['call_dur'] > 300]
tmp2 = tmp1.groupby('phone_no_m')['call_dur'].agg(voc_calltype_id_1_300s_cnt='count')
phone_no_m = phone_no_m.merge(tmp2, on='phone_no_m', how='left')
del tmp1, tmp2
gc.collect()

# # 主叫通话时长小于10s的次数的占比
# phone_no_m['voc_calltype_id_1_10s_rate'] = phone_no_m['voc_calltype_id_1_10s_cnt'] / phone_no_m['voc_calltype_id_1_cnt']

# 主叫通话时长小于30s的次数的占比
phone_no_m['voc_calltype_id_1_30s_rate'] = phone_no_m['voc_calltype_id_1_30s_cnt'] / phone_no_m['voc_calltype_id_1_cnt']

# 主叫通话时长小于60s的次数的占比
phone_no_m['voc_calltype_id_1_60s_rate'] = phone_no_m['voc_calltype_id_1_60s_cnt'] / phone_no_m['voc_calltype_id_1_cnt']

# 主叫通话时长大于300s（5分钟）的次数的占比
phone_no_m['voc_calltype_id_1_300s_rate'] = phone_no_m['voc_calltype_id_1_300s_cnt'] / phone_no_m['voc_calltype_id_1_cnt']


# 主叫通话次数的占比
phone_no_m["call_type_id_1_rate"] = phone_no_m['voc_calltype_id_1_cnt'] / phone_no_m['opposite_cnt']

# 主叫通话时所在地市变动的个数
tmp = df_calltype_id_1.groupby('phone_no_m')['city_name'].agg(city_name_nunique='nunique')
phone_no_m = phone_no_m.merge(tmp, on='phone_no_m', how='left')
del tmp
gc.collect()

# 主叫通话时所在区县变动的个数
tmp = df_calltype_id_1.groupby("phone_no_m")['county_name'].agg(county_name_nunique='nunique')
phone_no_m = phone_no_m.merge(tmp, on="phone_no_m", how="left")
del tmp
gc.collect()

# 主叫通话时所在地市_区县变动的个数
tmp = df_calltype_id_1.groupby("phone_no_m")['city_name_county_name'].agg(city_name_county_name_nunique='nunique')
phone_no_m = phone_no_m.merge(tmp, on="phone_no_m", how="left")
del tmp
gc.collect()

# 线上效果变差，不用
# # 主叫通话每天的通话次数
# calltype_id_1_voc_day_cnt_res = df_calltype_id_1.groupby(['phone_no_m', 'voc_day'])['phone_no_m'].count().unstack()
# for i in df_calltype_id_1['voc_day'].unique():
#     phone_no_m['calltype_id_1_voc_day{}_count'.format(i)] = phone_no_m['phone_no_m'].map(calltype_id_1_voc_day_cnt_res[i])

# del calltype_id_1_voc_day_cnt_res
# gc.collect()

0

In [25]:
"""
被叫通话
"""

df_calltype_id_2 = df_voc.loc[df_voc['calltype_id'] == 2, :].copy()

# 被叫通话的次数
tmp = df_calltype_id_2.groupby('phone_no_m')['imei_m'].agg(voc_calltype_id_2_cnt='count')
phone_no_m = phone_no_m.merge(tmp, on='phone_no_m', how='left')

# 被叫通话次数的占比
phone_no_m["call_type_id_2_rate"] = phone_no_m['voc_calltype_id_2_cnt'] / phone_no_m['opposite_cnt']

del tmp, df_calltype_id_2
gc.collect()

线上变差，删除
# 主叫次数/被叫次数
phone_no_m['voc_calltype_1/2'] = phone_no_m['voc_calltype_id_1_cnt'] / (phone_no_m['voc_calltype_id_2_cnt'] + 0.0001)

'\n被叫通话\n'

In [26]:
"""
呼叫转移
"""

df_calltype_id_3 = df_voc.loc[df_voc['calltype_id'] == 3, :].copy()

# 呼叫转移的次数
tmp = df_calltype_id_3.groupby('phone_no_m')['imei_m'].agg(voc_calltype_id_3_cnt='count')
phone_no_m = phone_no_m.merge(tmp, on='phone_no_m', how='left')

# 呼叫转移次数的占比
phone_no_m["call_type_id_3_rate"] = phone_no_m['voc_calltype_id_3_cnt'] / phone_no_m['opposite_cnt']

del tmp, df_calltype_id_3
gc.collect()

# 线上变差，删除
# # 主叫次数/呼叫转移次数
# phone_no_m['voc_calltype_1/3'] = phone_no_m['voc_calltype_id_1_cnt'] / (phone_no_m['voc_calltype_id_3_cnt'] + 0.0001)

0

In [27]:
"""
与对端通话统计
"""

# 与对端通话次数，与对端通话时长
tmp = df_voc.groupby(['phone_no_m', 'opposite_no_m'])['call_dur'].agg(call_count='count',
                                                                      call_sum='sum')

# 与对端通话次数的统计量
phone2opposite = tmp.groupby('phone_no_m')['call_count'].agg(phone2opposite_cnt_mean='mean',
                                                             phone2opposite_cnt_median='median',
                                                             phone2opposite_cnt_min='min',
                                                             phone2opposite_cnt_max='max',
                                                             phone2opposite_cnt_std='std')
phone_no_m = phone_no_m.merge(phone2opposite, on='phone_no_m', how='left')
del phone2opposite
gc.collect()


# 与对端总通话时长的统计量
phone2opposite = tmp.groupby('phone_no_m')['call_sum'].agg(phone2opposite_call_dur_mean='mean',
                                                           phone2opposite_call_dur_median='median',
                                                           phone2opposite_call_dur_min='min',
                                                           phone2opposite_call_dur_max='max',
                                                           phone2opposite_call_dur_std='std')
phone_no_m = phone_no_m.merge(phone2opposite, on='phone_no_m', how='left')
del phone2opposite, tmp
gc.collect()

0

In [28]:
"""
通话时长的统计
"""

# 通话时长的统计量
tmp = df_voc.groupby('phone_no_m')['call_dur'].agg(call_dur_mean='mean',
                                                   call_dur_median='median',
                                                   call_dur_max='max',
                                                   call_dur_min='min',
                                                   call_dur_std='std',
                                                   call_dur_sum='sum')
phone_no_m = phone_no_m.merge(tmp, on='phone_no_m', how='left')
del tmp
gc.collect()

0

In [29]:
"""
收费号码位置变动
"""

# 收费号码所在地市的个数
tmp = df_voc.groupby('phone_no_m')['city_name'].agg(city_name_nunique='nunique')
phone_no_m = phone_no_m.merge(tmp, on='phone_no_m', how='left')

# 收费号码所在区县的个数
tmp = df_voc.groupby('phone_no_m')['county_name'].agg(county_name_nunique='nunique')
phone_no_m = phone_no_m.merge(tmp, on="phone_no_m", how='left')

# 收费号码所在地市_区县的个数
tmp = df_voc.groupby('phone_no_m')['city_name_county_name'].agg(city_name_county_name_nunique='nunique')
phone_no_m = phone_no_m.merge(tmp, on="phone_no_m", how='left')

# 收费号码通话类型的个数
tmp = df_voc.groupby('phone_no_m')['calltype_id'].agg(calltype_id_unique='nunique')
phone_no_m = phone_no_m.merge(tmp, on='phone_no_m', how='left')

del tmp
gc.collect()

0

In [30]:
"""
通话时间点的偏好
"""

# hour通话次数最高
tmp = df_voc.groupby('phone_no_m')['voc_hour'].agg(voc_hour_mode=lambda x: stats.mode(x)[0][0],        # 频次最高的元素
                                                   voc_hour_mode_count=lambda x: stats.mode(x)[1][0],  # 频次最高的元素的频次
                                                   voc_hour_nunique='nunique')
phone_no_m = phone_no_m.merge(tmp, on='phone_no_m', how='left')

# day通话次数最高
tmp = df_voc.groupby('phone_no_m')['voc_day'].agg(voc_day_mode=lambda x: stats.mode(x)[0][0],          # 频次最高的元素
                                                  voc_day_mode_count=lambda x: stats.mode(x)[1][0],    # 频次最高的元素的频次
                                                  voc_day_nunique='nunique')                           # 一个月有多少天打了电话
phone_no_m = phone_no_m.merge(tmp, on='phone_no_m', how='left')

del tmp
gc.collect()


# dayofweek通话次数最高
tmp = df_voc.groupby('phone_no_m')['voc_dayofweek'].agg(voc_dayofweek_mode=lambda x: stats.mode(x)[0][0],          # 频次最高的元素
                                                        voc_dayofweek_mode_count=lambda x: stats.mode(x)[1][0],    # 频次最高的元素的频次
                                                        voc_dayofweek_nunique='nunique')                           
phone_no_m = phone_no_m.merge(tmp, on='phone_no_m', how='left')

del tmp
gc.collect()

0

In [31]:
# 每天通话次数的均值、标准差
tmp1 = df_voc.groupby(['phone_no_m', 'voc_day'])['phone_no_m'].agg(voc_day_cnt='count')
tmp2 = tmp1.groupby('phone_no_m')['voc_day_cnt'].agg(voc_day_cnt_mean='mean',
                                                     voc_day_cnt_std='std')
phone_no_m = phone_no_m.merge(tmp2, on='phone_no_m', how='left')

del tmp1, tmp2
gc.collect()

20

In [32]:
# 每天的通话次数
voc_day_cnt_res = df_voc.groupby(['phone_no_m', 'voc_day'])['phone_no_m'].count().unstack()
for i in df_voc['voc_day'].unique():
    phone_no_m['voc_day{}_count'.format(i)] = phone_no_m['phone_no_m'].map(voc_day_cnt_res[i])

    
# 每天的通话人数
voc_day_nunique_res = df_voc.groupby(['phone_no_m', 'voc_day'])['opposite_no_m'].nunique().unstack()
for i in df_voc['voc_day'].unique():
    phone_no_m['voc_day{}_nunique'.format(i)] = phone_no_m['phone_no_m'].map(voc_day_nunique_res[i])

    
# 每天的通话时长
voc_day_call_dur_res = df_voc.groupby(['phone_no_m', 'voc_day'])['call_dur'].sum().unstack()
for i in df_voc['voc_day'].unique():
    phone_no_m['voc_day{}_call_dur_sum'.format(i)] = phone_no_m['phone_no_m'].map(voc_day_call_dur_res[i])



# 每小时的通话次数
voc_hour_cnt_res = df_voc.groupby(['phone_no_m', 'voc_hour'])['phone_no_m'].count().unstack()
for i in df_voc['voc_hour'].unique():
    phone_no_m['voc_hour{}_count'.format(i)] = phone_no_m['phone_no_m'].map(voc_hour_cnt_res[i])
    
# 每小时的通话人数
voc_hour_nunique_res = df_voc.groupby(['phone_no_m', 'voc_hour'])['opposite_no_m'].nunique().unstack()
for i in df_voc['voc_hour'].unique():
    phone_no_m['voc_hour{}_nunique'.format(i)] = phone_no_m['phone_no_m'].map(voc_hour_nunique_res[i])
    

# 每小时的通话时长
voc_hour_call_dur_res = df_voc.groupby(['phone_no_m', 'voc_hour'])['call_dur'].sum().unstack()
for i in df_voc['voc_hour'].unique():
    phone_no_m['voc_hour{}_call_dur_sum'.format(i)] = phone_no_m['phone_no_m'].map(voc_hour_call_dur_res[i])

    

# 每周几的通话次数
voc_dayofweek_cnt_res = df_voc.groupby(['phone_no_m', 'voc_dayofweek'])['phone_no_m'].count().unstack()
for i in df_voc['voc_dayofweek'].unique():
    phone_no_m['voc_dayofweek{}_count'.format(i)] = phone_no_m['phone_no_m'].map(voc_dayofweek_cnt_res[i])
    
# 每周几的通话人数
voc_dayoffweek_nunique_res = df_voc.groupby(['phone_no_m', 'voc_dayofweek'])['opposite_no_m'].nunique().unstack()
for i in df_voc['voc_dayofweek'].unique():
    phone_no_m['voc_dayofweek{}_nunique'.format(i)] = phone_no_m['phone_no_m'].map(voc_dayoffweek_nunique_res[i])
    

# 每周几的通话时长
voc_dayofweek_call_dur_res = df_voc.groupby(['phone_no_m', 'voc_dayofweek'])['call_dur'].sum().unstack()
for i in df_voc['voc_dayofweek'].unique():
    phone_no_m['voc_dayofweek{}_call_dur_sum'.format(i)] = phone_no_m['phone_no_m'].map(voc_dayofweek_call_dur_res[i])

In [33]:
phone_no_m.to_hdf('../input/voc_features.h5', 'df', index=False)
del phone_no_m
gc.collect()

75

### 被叫通话

In [1]:
opposite_no_m = df_voc[['opposite_no_m']].copy()
opposite_no_m = opposite_no_m.drop_duplicates(subset=['opposite_no_m'], keep='last')

NameError: name 'df_user' is not defined

In [ ]:
"""
被叫通话
"""

df_calltype_id_2 = df_voc.loc[df_voc['calltype_id'] == 2, :].copy()

# 被叫通话次数，被叫通话使用的手机个数
tmp = df_calltype_id_2.groupby('opposite_no_m')['imei_m'].agg(voc_calltype_id_2_cnt='count',
                                                              imeis_2='nunique')
phone_no_m = phone_no_m.merge(tmp, left_on='phone_no_m', right_on='opposite_no_m', how='left')
del tmp
gc.collect()

# 被叫通话时长
tmp = df_calltype_id_2.groupby('opposite_no_m')['call_dur'].agg(voc_calltype_id_2_call_dur_sum='sum')
phone_no_m = phone_no_m.merge(tmp, left_on='phone_no_m', right_on='opposite_no_m', how='left')
del tmp
gc.collect()

# 被叫通话时长小于10s的次数
# tmp1 = df_calltype_id_1[df_calltype_id_1['call_dur'] < 10]
# tmp2 = tmp1.groupby('phone_no_m')['call_dur'].agg(voc_calltype_id_1_10s_cnt='count')
# phone_no_m = phone_no_m.merge(tmp2, on='phone_no_m', how='left')
# del tmp1, tmp2
# gc.collect()

# 被叫通话时长小于30s的次数
tmp1 = df_calltype_id_2[df_calltype_id_2['call_dur'] < 30]
tmp2 = tmp1.groupby('opposite_no_m')['call_dur'].agg(voc_calltype_id_2_30s_cnt='count')
phone_no_m = phone_no_m.merge(tmp2, left_on='phone_no_m', right_on='opposite_no_m', how='left')
del tmp1, tmp2
gc.collect()

# 被叫通话时长小于60s的次数
tmp1 = df_calltype_id_2[df_calltype_id_2['call_dur'] < 60]
tmp2 = tmp1.groupby('opposite_no_m')['call_dur'].agg(voc_calltype_id_2_60s_cnt='count')
phone_no_m = phone_no_m.merge(tmp2, left_on='phone_no_m', right_on='opposite_no_m', how='left')
del tmp1, tmp2
gc.collect()

# 被叫通话时长大于300s（5分钟）的次数
tmp1 = df_calltype_id_2[df_calltype_id_2['call_dur'] > 300]
tmp2 = tmp1.groupby('opposite_no_m')['call_dur'].agg(voc_calltype_id_2_300s_cnt='count')
phone_no_m = phone_no_m.merge(tmp2, left_on='phone_no_m', right_on='opposite_no_m', how='left')
del tmp1, tmp2
gc.collect()

# # 被叫通话时长小于10s的次数的占比
# phone_no_m['voc_calltype_id_1_10s_rate'] = phone_no_m['voc_calltype_id_1_10s_cnt'] / phone_no_m['voc_calltype_id_1_cnt']

# 被叫通话时长小于30s的次数的占比
phone_no_m['voc_calltype_id_2_30s_rate'] = phone_no_m['voc_calltype_id_2_30s_cnt'] / phone_no_m['voc_calltype_id_2_cnt']

# 被叫通话时长小于60s的次数的占比
phone_no_m['voc_calltype_id_2_60s_rate'] = phone_no_m['voc_calltype_id_2_60s_cnt'] / phone_no_m['voc_calltype_id_2_cnt']

# 被叫通话时长大于300s（5分钟）的次数的占比
phone_no_m['voc_calltype_id_2_300s_rate'] = phone_no_m['voc_calltype_id_2_300s_cnt'] / phone_no_m['voc_calltype_id_2_cnt']


# 被叫通话次数的占比
phone_no_m["call_type_id_2_rate"] = phone_no_m['voc_calltype_id_2_cnt'] / phone_no_m['opposite_cnt']

# # 被叫通话时所在地市变动的个数
# tmp = df_calltype_id_2.groupby('opposite_no_m')['city_name'].agg(city_name_nunique='nunique')
# phone_no_m = phone_no_m.merge(tmp, left_on='phone_no_m', right_on='opposite_no_m', how='left')
# del tmp
# gc.collect()

# # 被叫通话时所在区县变动的个数
# tmp = df_calltype_id_2.groupby('opposite_no_m')['county_name'].agg(county_name_nunique='nunique')
# phone_no_m = phone_no_m.merge(tmp, left_on='phone_no_m', right_on='opposite_no_m', how='left')
# del tmp
# gc.collect()

# # 被叫通话时所在地市_区县变动的个数
# tmp = df_calltype_id_2.groupby('opposite_no_m')['city_name_county_name'].agg(city_name_county_name_nunique='nunique')
# phone_no_m = phone_no_m.merge(tmp, left_on='phone_no_m', right_on='opposite_no_m', how='left')
# del tmp
# gc.collect()

# 线上效果变差，不用
# # 被叫通话每天的通话次数
# calltype_id_1_voc_day_cnt_res = df_calltype_id_1.groupby(['phone_no_m', 'voc_day'])['phone_no_m'].count().unstack()
# for i in df_calltype_id_1['voc_day'].unique():
#     phone_no_m['calltype_id_1_voc_day{}_count'.format(i)] = phone_no_m['phone_no_m'].map(calltype_id_1_voc_day_cnt_res[i])

# del calltype_id_1_voc_day_cnt_res
# gc.collect()

## sms表

In [34]:
train_sms = pd.read_hdf('../input/train_sms.h5')
test_sms = pd.read_hdf('../input/test_sms.h5')

In [35]:
df_sms = pd.concat([train_sms, test_sms])

del train_sms, test_sms
gc.collect()

106

In [36]:
df_sms['sms_day'] = df_sms['request_datetime'].astype('datetime64').dt.day
df_sms['sms_hour'] = df_sms['request_datetime'].astype('datetime64').dt.hour
df_sms['sms_dayofweek'] = df_sms['request_datetime'].astype('datetime64').dt.dayofweek

In [37]:
phone_no_m = df_sms[["phone_no_m"]].copy()
phone_no_m = phone_no_m.drop_duplicates(subset=['phone_no_m'], keep='last')

In [38]:
# 短信次数，短信人数
tmp = df_sms.groupby('phone_no_m')['opposite_no_m'].agg(sms_cnt='count', sms_nunique='nunique')

# 人均短信次数
tmp['sms_cnt_per_capita'] = tmp['sms_cnt'] / (tmp['sms_nunique'] + 0.0001)
phone_no_m = phone_no_m.merge(tmp, on='phone_no_m', how='left')

del tmp
gc.collect()

0

In [39]:
"""
短信上行，短信下行
"""

# 短信上行
df_sms_calltype1 = df_sms[df_sms['calltype_id'] == 1].copy()
tmp = df_sms_calltype1.groupby('phone_no_m')['calltype_id'].agg(sms_calltype1_cnt="count")
phone_no_m = phone_no_m.merge(tmp, on='phone_no_m', how='left')

# 短信上行比例
phone_no_m['sms_calltype1_rate'] = phone_no_m['sms_calltype1_cnt'] / phone_no_m['sms_cnt']

del tmp, df_sms_calltype1

# 短信下行
df_sms_calltype2 = df_sms[df_sms['calltype_id'] == 2].copy()
tmp = df_sms_calltype2.groupby('opposite_no_m')['calltype_id'].agg(sms_calltype2_cnt="count")
phone_no_m = phone_no_m.merge(tmp, left_on='phone_no_m', right_on='opposite_no_m', how='left')

# 短信下行比例
phone_no_m['sms_calltype2_rate'] = phone_no_m['sms_calltype2_cnt'] / phone_no_m['sms_cnt']

# 短信上行/短信下行
phone_no_m['type1_rate/type2_rate'] = phone_no_m['sms_calltype1_cnt'] / (phone_no_m['sms_calltype2_cnt'] + 0.00001)

# 删除，效果变差
# # 短信下行/短信上行
# phone_no_m['type2_rate/type1_rate'] = phone_no_m['sms_calltype2_cnt'] / (phone_no_m['sms_calltype1_cnt'] + 0.00001)

del tmp, df_sms_calltype2
gc.collect()

0

In [40]:
"""
短信时间点的偏好
"""

tmp = df_sms.groupby('phone_no_m')['sms_hour'].agg(sms_hour_mode=lambda x: stats.mode(x)[0][0],        # 频次最高的元素
                                                   sms_hour_mode_count=lambda x: stats.mode(x)[1][0],  # 频次最高的元素的频次
                                                   sms_hour_nunique='nunique')                          # 一个月有多少个小时发了短信
phone_no_m = phone_no_m.merge(tmp, on='phone_no_m', how='left')

del tmp
gc.collect()

tmp = df_sms.groupby('phone_no_m')['sms_day'].agg(sms_day_mode=lambda x: stats.mode(x)[0][0],          # 频次最高的元素
                                                  sms_day_mode_count=lambda x: stats.mode(x)[1][0],    # 频次最高的元素的频次
                                                  sms_day_nunique='nunique')                            # 一个月有多少天发了短信
phone_no_m = phone_no_m.merge(tmp, on='phone_no_m', how='left')

del tmp
gc.collect()


tmp = df_sms.groupby('phone_no_m')['sms_dayofweek'].agg(sms_dayofweek_mode=lambda x: stats.mode(x)[0][0],          # 频次最高的元素
                                                        sms_dayofweek_mode_count=lambda x: stats.mode(x)[1][0],    # 频次最高的元素的频次
                                                        sms_dayofweek_nunique='nunique')                            # 一个月有多少天发了短信
phone_no_m = phone_no_m.merge(tmp, on='phone_no_m', how='left')

del tmp
gc.collect()

0

In [41]:
# 每天的短信次数
sms_day_res = df_sms.groupby(['phone_no_m', 'sms_day'])['phone_no_m'].count().unstack()
for i in df_sms['sms_day'].unique():
    phone_no_m['sms_day{}_count'.format(i)] = phone_no_m['phone_no_m'].map(sms_day_res[i])


# 每小时的短信次数
sms_hour_res = df_sms.groupby(['phone_no_m', 'sms_hour'])['phone_no_m'].count().unstack()
for i in df_sms['sms_hour'].unique():
    phone_no_m['sms_hour{}_count'.format(i)] = phone_no_m['phone_no_m'].map(sms_hour_res[i])
    
    
# 每周几的短信次数
sms_dayofweek_res = df_sms.groupby(['phone_no_m', 'sms_dayofweek'])['phone_no_m'].count().unstack()
for i in df_sms['sms_dayofweek'].unique():
    phone_no_m['sms_weekofday{}_count'.format(i)] = phone_no_m['phone_no_m'].map(sms_dayofweek_res[i])

In [42]:
phone_no_m.to_hdf('../input/sms_features.h5', 'df', index=False)

del phone_no_m
gc.collect()

64

## app表

In [43]:
train_app = pd.read_hdf('../input/train_app.h5')
test_app = pd.read_hdf('../input/test_app.h5')

In [44]:
df_app = pd.concat([train_app, test_app])

del train_app, test_app
gc.collect()

106

In [45]:
phone_no_m = df_app[['phone_no_m']].copy()
phone_no_m = phone_no_m.drop_duplicates(subset=['phone_no_m'], keep='last')

In [46]:
# APP数
tmp = df_app.groupby('phone_no_m')['busi_name'].agg(busi_cnt='count')
phone_no_m = phone_no_m.merge(tmp, on='phone_no_m', how='left')
del tmp
gc.collect()

40

In [47]:
"""
流量统计
"""
tmp = df_app.groupby("phone_no_m")["flow"].agg(flow_mean='mean',
                                               flow_median='median',
                                               flow_min='min',
                                               flow_max='max',
                                               flow_std='std',
                                               flow_sum='sum',
                                               flow_skew='skew')
phone_no_m = phone_no_m.merge(tmp, on='phone_no_m', how='left')

del tmp
gc.collect()

20

In [48]:
phone_no_m.to_hdf('../input/app_features.h5', 'df', index=False)

del phone_no_m
gc.collect()

75

## 读取数据，建模

In [49]:
df_user = reduce_mem_usage(pd.read_hdf('../input/user_features.h5'))
df_voc = reduce_mem_usage(pd.read_hdf('../input/voc_features.h5'))
df_sms = reduce_mem_usage(pd.read_hdf('../input/sms_features.h5'))
# df_app = reduce_mem_usage(pd.read_hdf('../input/app_features.h5'))

Memory usage of dataframe is 4042896.00 MB
Memory usage after optimization is: 1858428.00 MB
Decreased by 54.0%
Memory usage of dataframe is 13331632.00 MB
Memory usage after optimization is: 3441516.00 MB
Decreased by 74.2%
Memory usage of dataframe is 4455000.00 MB
Memory usage after optimization is: 1292500.00 MB
Decreased by 71.0%


In [50]:
df_user.shape, df_voc.shape, df_sms.shape #, df_app.shape

((8151, 91), (6788, 246), (6875, 80))

In [51]:
df = df_user.merge(df_voc, on='phone_no_m', how='left')
df = df.merge(df_sms, on='phone_no_m', how='left')
# df = df.merge(df_app, on='phone_no_m', how='left')

del df_user, df_voc, df_sms#, df_app
gc.collect()

0

In [52]:
df.shape

(8151, 415)

In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8151 entries, 0 to 8150
Columns: 415 entries, phone_no_m to sms_weekofday5_count
dtypes: float16(342), float32(10), float64(39), int16(11), int8(12), object(1)
memory usage: 8.4+ MB


In [54]:
df.isnull().sum().sort_values()

phone_no_m                                                 0
city_name_idcard_cnt*arpu_202004_nunique                   0
city_name_arpu_202004/idcard_cnt_max                       0
city_name_arpu_202004/idcard_cnt_min                       0
city_name_arpu_202004/idcard_cnt_median                    0
city_name_arpu_202004/idcard_cnt_mean                      0
city_name_arpu_202004/idcard_cnt_skew                      0
city_name_arpu_202004/idcard_cnt_nunique                   0
county_name_idcard_cnt_max                                 0
county_name_idcard_cnt_min                                 0
county_name_idcard_cnt_median                              0
county_name_idcard_cnt_mean                                0
county_name_idcard_cnt_nunique                             0
city_name_idcard_cnt*arpu_202004_mean                      0
county_name_arpu_202004_nunique                            0
county_name_arpu_202004/idcard_cnt_nunique                 0
city_name_county_name_id

In [55]:
df = df.fillna(df.mode().iloc[0,:])
# df.fillna(999999, inplace=True)

In [57]:
df.isnull().sum().sort_values()

phone_no_m                                              0
voc_hour17_nunique                                      0
voc_hour16_nunique                                      0
voc_hour8_nunique                                       0
voc_hour22_nunique                                      0
voc_hour13_nunique                                      0
voc_hour10_nunique                                      0
voc_hour20_nunique                                      0
voc_hour6_nunique                                       0
voc_hour14_nunique                                      0
voc_hour12_nunique                                      0
voc_hour7_nunique                                       0
voc_hour9_nunique                                       0
voc_hour15_nunique                                      0
voc_hour11_nunique                                      0
voc_hour18_nunique                                      0
voc_hour21_nunique                                      0
voc_hour4_coun

In [56]:
nmf = NMF(n_components=300,
          init='random',
          random_state=2020)

df_nmf = nmf.fit_transform(df.drop(['phone_no_m', 'label'], axis=1))
df_nmf['label'] = df['label']

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
# astype('category')会掉分，不做此处理
# for i in tqdm(cat_feat):
#     print(i)
#     df[i] = df[i].astype('category')

In [ ]:
df_nmf = df_nmf.replace(999999, np.nan)

In [ ]:
df_train = df_nmf[df_nmf.label.notna()]
df_test = df_nmf[df_nmf.label.isna()]

df_train.shape, df_test.shape

In [ ]:
fs = FeatureSelector(data=df_train.drop(['phone_no_m', 'label'], axis=1), labels=df_train['label'])

fs.identify_all(selection_params={'missing_threshold': 0.98,
                                  'correlation_threshold': 0.98, 
                                  'task': 'classification',
                                  'eval_metric': 'auc', 
                                  'cumulative_importance': 0.99})

In [ ]:
train_removed_all_once = fs.remove(methods='all')
# train_removed_all_once

In [ ]:
use_cols = train_removed_all_once.columns.to_list()

del train_removed_all_once
gc.collect()

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(df_train[use_cols], df_train['label'],
                                                      test_size=0.2,
                                                      random_state=2020)

In [ ]:
train_cols = [i for i in X_train.columns if i not in ['phone_no_m', 'label']]

In [ ]:
params = {'objective': 'binary',
          'boosting': 'gbdt',
          'metric': 'auc',
          'learning_rate': 0.1,
          'num_leaves': 31,
          'lambda_l1': 0.1,
          'lambda_l2': 0,
          'min_data_in_leaf': 20,
          'is_unbalance': True,
          'max_depth': -1,
          'seed': 2020}

In [ ]:
useful_cols = []
useless_cols = []

for i in train_cols:
    print(i)
    
    lgb_train = lgb.Dataset(X_train[[i]].values, y_train) 
    lgb_valid= lgb.Dataset(X_valid[[i]].values, y_valid, reference=lgb_train)
    lgb_test = lgb.train(params,
                         lgb_train,
                         num_boost_round=1000,
                         valid_sets=[lgb_valid, lgb_train],
                         early_stopping_rounds=50,
                         verbose_eval=20)
    
    print('*' * 5)
    print(lgb_test.best_score['valid_0']['auc'])
    if lgb_test.best_score['valid_0']['auc'] > 0.52:
        useful_cols.append(i)
    else:
        useless_cols.append(i)
    print('*' * 20)
    print('\n')

In [ ]:
print(useless_cols)

In [ ]:
lgb_train = lgb.Dataset(X_train[useful_cols].values, y_train) 

lgb_valid= lgb.Dataset(X_valid[useful_cols].values, y_valid, reference=lgb_train)  

print('Start training...')

lgb_val_0 = lgb.train(params,
                      lgb_train,
                      num_boost_round=10000,
                      valid_sets=[lgb_valid, lgb_train],
                      early_stopping_rounds=100,
                      verbose_eval=10)

print('Done!')

In [ ]:
# 验证集结果
X_valid['prob'] = lgb_val_0.predict(X_valid[useful_cols])
X_valid['pred'] = np.where(X_valid['prob'] > 0.4735, 1, 0)

f1_04735 = np.round(f1_score(y_valid, X_valid['pred']), 4)
auc_04735 = roc_auc_score(y_valid, X_valid['prob'])

print('f1_04735: ', f1_04735)
print('auc_04735: ', auc_04735)

In [ ]:
lgb_train_all = lgb.Dataset(df_train[useful_cols].values, df_train['label'])   

print('Start training...')

lgb_model = lgb.train(params,
                      lgb_train_all,
                      num_boost_round=lgb_val_0.best_iteration + 20)

print('Done!')

In [ ]:
df_test['label'] = np.where(lgb_model.predict(df_test[useful_cols]) > 0.4735, 1, 0)
df_test[['phone_no_m', 'label']].to_csv('../sub/sub_{}_{}.csv'.format(time.strftime('%Y%m%d'), f1_04735), index=False)